In [1]:
# 1) Импорт всех необходимых библиотек
from rich import print
import matplotlib.pyplot as plt
import numpy as np
import rclpy
from rclpy.wait_for_message import wait_for_message
from cv_bridge import CvBridge
from sensor_msgs.msg import NavSatFix, MagneticField
from geometry_msgs.msg import Twist
import math

# 2) Инциализация `rclpy` и создания node для наблюдения за сообщениями
if not rclpy.ok():
    rclpy.init()
    node = rclpy.create_node("simple_node")

def get_angle(dx,dy):
    # get angle relativ X-axes
    pre_angle = math.atan2(dx,dy)
    if pre_angle>=0:
        return pre_angle
    else:
        return (math.pi*2 + pre_angle)

    

def get_gps():
    msg_nav = wait_for_message(NavSatFix,node,"/gps/nav",time_to_wait=3)
    msg_mag = wait_for_message(MagneticField,node,"/gps/magnetometer",time_to_wait=3)
    
    if msg_nav[0] & msg_mag[0]:
        return {
            "nav":{
                "latitude":msg_nav[1].latitude,
                "longitude":msg_nav[1].longitude
            },
            "angle":get_angle(msg_mag[1].magnetic_field.x, msg_mag[1].magnetic_field.y)
        } 
    else:
        raise Exception("Timeout")

def get_control_angle(point):
    gps_now = get_gps()
    nav_now = gps_now["nav"]
    robot_angle = gps_now["angle"]
    dx = point['latitude'] - nav_now['latitude'] 
    dy = point['longitude'] - nav_now['longitude']
    point_angle = get_angle(dx, dy)

    if point_angle>robot_angle:
        left = point_angle - robot_angle
        right = (math.pi*2 - point_angle)+ robot_angle
    elif point_angle<robot_angle:
        left = (math.pi*2 - robot_angle)+ point_angle
        right = robot_angle - point_angle
    else:
        left = right = math.pi

    if right < left:
        return -right
    elif left < right:
        return left
    else:
        return 0.0


cmd_vel_pub = node.create_publisher(Twist, "/cmd_vel", 5)

In [2]:
goals = [{'latitude': 1.4487750602732721e-05, 'longitude': 2.5566841833230843e-05},
         {'latitude': 1.4487614763096712e-05, 'longitude': -4.378236310666495e-06},
         {'latitude': 2.9282975907335024e-05, 'longitude': -1.897351782894222e-05},
         {'latitude': 1.3025943609366113e-05, 'longitude': -3.247089525617746e-05},
         {'latitude': 1.302537761083063e-05, 'longitude': -0.00015560581435193746},
         {'latitude': -2.0312146866997443e-05, 'longitude': -0.00015560605422798248},
         {'latitude': -2.2080421019494802e-05, 'longitude': -0.000134269379868968},
         {'latitude': -2.2080300273181826e-05, 'longitude': -9.357683329355574e-05}]
doing_work_index = 0


In [3]:
while(1):
    
    msg = Twist()
    if doing_work_index >= len(goals):
        
        cmd_vel_pub.publish(msg)
        break
    goal = goals[doing_work_index]
    now_gps = get_gps()["nav"]
    distance = math.sqrt((goal["latitude"] - now_gps["latitude"])**2+ (goal["longitude"] - now_gps["longitude"])**2)
    if distance<2.0e-06:
        doing_work_index +=1
        
    
    angle = get_control_angle(goal)
    angle_moving_treshold = 0.5 
    if angle < angle_moving_treshold:
        msg.linear.x = 0.5
        msg.angular.z = angle*2
    else:
        msg.linear.x = 0.0
        msg.angular.z = 0.5 if angle>0 else -0.5
    
    cmd_vel_pub.publish(msg)
# print(msg.linear.z)

Exception: Timeout

In [20]:
# 1) Импорт всех необходимых библиотек
from rich import print
import matplotlib.pyplot as plt
import numpy as np
import rclpy
from rclpy.wait_for_message import wait_for_message
from cv_bridge import CvBridge
from sensor_msgs.msg import NavSatFix, MagneticField
from geometry_msgs.msg import Twist
import math

# 2) Инциализация `rclpy` и создания node для наблюдения за сообщениями
if not rclpy.ok():
    rclpy.init()
    node = rclpy.create_node("simple_node")

def get_angle(dx,dy):
    # get angle relativ X-axes
    pre_angle = math.atan2(dx,dy)
    if pre_angle>=0:
        return pre_angle
    else:
        return (math.pi*2 + pre_angle)

    

def get_gps():
    msg_nav = wait_for_message(NavSatFix,node,"/gps/nav",time_to_wait=3)
    msg_mag = wait_for_message(MagneticField,node,"/gps/magnetometer",time_to_wait=3)
    
    if msg_nav[0] & msg_mag[0]:
        return {
            "nav":{
                "latitude":msg_nav[1].latitude,
                "longitude":msg_nav[1].longitude
            },
            "angle":get_angle(msg_mag[1].magnetic_field.x, msg_mag[1].magnetic_field.y)
        } 
    else:
        raise Exception("Timeout")

def get_control_angle(point):
    gps_now = get_gps()
    nav_now = gps_now["nav"]
    robot_angle = gps_now["angle"]
    dx = point['latitude'] - nav_now['latitude'] 
    dy = point['longitude'] - nav_now['longitude']
    point_angle = get_angle(dx, dy)

    if point_angle>robot_angle:
        left = point_angle - robot_angle
        right = (math.pi*2 - point_angle)+ robot_angle
    elif point_angle<robot_angle:
        left = (math.pi*2 - robot_angle)+ point_angle
        right = robot_angle - point_angle
    else:
        left = right = math.pi

    if right < left:
        return -right
    elif left < right:
        return left
    else:
        return 0.0


cmd_vel_pub = node.create_publisher(Twist, "/cmd_vel", 5)

goals = [{'latitude': 1.4487750602732721e-05, 'longitude': 2.5566841833230843e-05},
         {'latitude': 1.4487614763096712e-05, 'longitude': -4.378236310666495e-06},
         {'latitude': 2.9282975907335024e-05, 'longitude': -1.897351782894222e-05},
         {'latitude': 1.3025943609366113e-05, 'longitude': -3.247089525617746e-05},
         {'latitude': 1.302537761083063e-05, 'longitude': -0.00015560581435193746},
         {'latitude': -2.0312146866997443e-05, 'longitude': -0.00015560605422798248},
         {'latitude': -2.2080421019494802e-05, 'longitude': -0.000134269379868968},
         {'latitude': -2.2080300273181826e-05, 'longitude': -9.357683329355574e-05}]
doing_work_index = 0


while(1):
    
    msg = Twist()
    if doing_work_index >= len(goals):
        
        cmd_vel_pub.publish(msg)
        break
    goal = goals[doing_work_index]
    now_gps = get_gps()["nav"]
    distance = math.sqrt((goal["latitude"] - now_gps["latitude"])**2+ (goal["longitude"] - now_gps["longitude"])**2)
    if distance<2.0e-06:
        doing_work_index +=1
        
    
    angle = get_control_angle(goal)
    angle_moving_treshold = 0.5 
    if angle < angle_moving_treshold:
        msg.linear.x = 0.5
        msg.angular.z = angle*2
    else:
        msg.linear.x = 0.0
        msg.angular.z = 0.5 if angle>0 else -0.5
    
    cmd_vel_pub.publish(msg)
# print(msg.linear.z)

[{'latitude': 1.4487750602732721e-05, 'longitude': 2.5566841833230843e-05},
 {'latitude': 1.4487614763096712e-05, 'longitude': -4.378236310666495e-06},
 {'latitude': 2.9282975907335024e-05, 'longitude': -1.897351782894222e-05},
 {'latitude': 1.3025943609366113e-05, 'longitude': -3.247089525617746e-05},
 {'latitude': 1.302537761083063e-05, 'longitude': -0.00015560581435193746},
 {'latitude': -2.0312146866997443e-05, 'longitude': -0.00015560605422798248},
 {'latitude': -2.2080421019494802e-05, 'longitude': -0.000134269379868968},
 {'latitude': -2.2080300273181826e-05, 'longitude': -9.357683329355574e-05}]